<a href="https://colab.research.google.com/github/Digitall-Ivan/Data_Analytics_course/blob/main/Ivan_Zakorchevnyi_HW_14_4_%D0%90_%D0%92_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [2]:
import numpy as np
from scipy.stats import norm

# Визначив вхідні дані
p1 = 0.19   # базове утримання
p2 = 0.20   # після змін (+1%)
alpha = 0.05
power = 0.8
beta = 1 - power

# Отримав Z-значення
z_alpha = norm.ppf(1 - alpha/2)   # двосторонній тест
z_beta = norm.ppf(power)

# Порахував середню пропорцію
p = (p1 + p2) / 2

# Зробив розрахунок по формулі для розміру вибірки
num = (z_alpha * np.sqrt(2 * p * (1 - p)) + z_beta * np.sqrt(p1 * (1 - p1) + p2 * (1 - p2))) ** 2
den = (p2 - p1) ** 2
n_per_group = int(np.ceil(num / den))
total_n = n_per_group * 2

{"n_per_group": n_per_group, "total_n": total_n}


{'n_per_group': 24641, 'total_n': 49282}

Висновок: Мінімальна кількість користувачів у кожній групі: 24 641 або сумарно для проведення A/B тесту - не менше 49 282 користувачів. Тобто, щоб оцінити ефект у +1% утримання при базі 19% з потужністю 80% та    α=0.05, потрібно буде зібрати дані приблизно 50 тисяч користувачів.

2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
import pandas as pd

# Зчитав дані з файлу
df = pd.read_csv("/content/drive/MyDrive/I LOVE DATA EDU COURSE/RAWDATA/cookie_cats.csv")

# Визначив середнє значення retention_7 по версіях гри
retention_means = df.groupby("version")["retention_7"].mean()

# Вивів результат та записав в словник
retention_means.to_dict()

{'gate_30': 0.19020134228187918, 'gate_40': 0.18200004396667327}

Висновок: Середні значення утримання на 7-й день (retention_7) по версіях гри:

*   gate_30 → 19.02%
*   gate_40 → 18.20%

Формулювання гіпотез

H₀: Утримання на 7-й день однакове в обох версіях (нема статистично значущої різниці).

H₁: Версія з воротами на рівні 30 (gate_30) дає краще утримання через 7 днів, ніж версія з воротами на рівні 40 (gate_40).

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [6]:
import csv
from collections import Counter
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Зробив підрахунок успіхів (TRUE) та розмірів вибірки по версіях
successes = Counter()
totals = Counter()

with open("/content/drive/MyDrive/I LOVE DATA EDU COURSE/RAWDATA/cookie_cats.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        version = row["version"]
        val = row["retention_7"].strip().upper()
        if val in ["TRUE", "FALSE"]:
            successes[version] += (val == "TRUE")
            totals[version] += 1

# Зібрав дані для тесту
count = [successes["gate_30"], successes["gate_40"]]
nobs = [totals["gate_30"], totals["gate_40"]]

# Визначив Z-тест (двосторонній)
z_stat, p_value = proportions_ztest(count, nobs, alternative="two-sided")

# Розрахував довірчі інтервали для кожної групи
ci_30 = proportion_confint(successes["gate_30"], totals["gate_30"], alpha=0.05, method="normal")
ci_40 = proportion_confint(successes["gate_40"], totals["gate_40"], alpha=0.05, method="normal")

{
    "z_stat": round(float(z_stat), 4),
    "p_value": round(float(p_value), 4),
    "CI_gate_30": (round(ci_30[0], 4), round(ci_30[1], 4)),
    "CI_gate_40": (round(ci_40[0], 4), round(ci_40[1], 4))
}

{'z_stat': 3.1644,
 'p_value': 0.0016,
 'CI_gate_30': (0.1866, 0.1938),
 'CI_gate_40': (0.1785, 0.1855)}

Висновок: Оскільки 𝑝-𝑣𝑎𝑙𝑢𝑒 = 0.0016 < 0.05, ми відхиляємо H₀. Це означає, що є статистично значуща різниця між утриманням у групах.Довірчі інтервали для gate_30 та gate_40 не перетинаються:


*   gate_30 ≈ 18.7% – 19.4%
*   gate_40 ≈ 17.9% – 18.6%

Це додатково може підтверджувати, що різниця стабільна, а не випадкова.

Отже, версія з воротами на рівні 30 (gate_30) показує краще 7-денне утримання, ніж версія з воротами на рівні 40. Відповідно, переміщення воріт з рівня 30 на рівень 40 негативно відзначилось на поведінці гравців.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


Крок 1. Формулювання гіпотез

*   H₀ (нульова гіпотеза): Версія гри (gate_30 чи gate_40) та утримання на 7-й день незалежні (різниці в утриманні немає).
*   H₁ (альтернативна гіпотеза): Версія гри та утримання на 7-й день залежні (різниця є).

Крок 2 Обчислення значень Хі-квадрат:

In [7]:
import numpy as np
from scipy.stats import chi2_contingency

# Введи кількість успіхів і загальний розмір вибірки для обох груп
success_gate30 = 1931   # TRUE у gate_30
n_gate30 = 10155
success_gate40 = 1820   # TRUE у gate_40
n_gate40 = 10000

# Формуємо таблицю спостережень
table = np.array([
    [success_gate30, n_gate30 - success_gate30],
    [success_gate40, n_gate40 - success_gate40]
])

chi2, p, dof, expected = chi2_contingency(table, correction=False)

print(f"Chi2 statistic: {chi2:.4f}")
print(f"p-value: {p:.4f}")


Chi2 statistic: 2.2109
p-value: 0.1370


Інтерпретація даних та загальний висновок:
при рівню значущості: α = 0.05, оскільки p = 0.1370>0.05, ми не можемо відхилити нульову гіпотезу. Це означає, що немає статистично значущої залежності між версією гри (gate_30 чи gate_40) та ймовірністю утримання гравця на 7-й день. Отже, результати суперечливі: можлива невелика різниця, але за Хі-квадрат тестом доказів недостатньо, щоб стверджувати про наявну залежність. З іншого боку, за результатами z-тесту ми бачили різницю в середніх пропорціях, тоді як тест Хі-квадрат показав, що ця різниця не є статистично значущою на рівні 5%. Таким чином, такий фактор як рівень воріт для даної гри можна вважати не надто сильним фактором, що сприяє утриманню. На мій погляд, є сенс тестувати інші фактори та знаходити більш значущі фактори для утримання гравців.